# Convolutional Neural Network

### Importing the libraries

In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [51]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

import datasets

In [84]:
#mount google drive to this notebook
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load the Training set

In [53]:
training_set = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/CNN/dataset/training_set', # path to training data
                                                           image_size=(64,64), # resize all images to 128x128
                                                           batch_size=32)  # number of images per batch

# Store class_indices before caching and prefetching
class_names = training_set.class_names

Found 8060 files belonging to 2 classes.


### Load the validation set

In [54]:
# here we want to feature scaling this test
validation_set = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/CNN/dataset/test_set', # path to training data
                                                                    image_size=(64,64), # resize all images
                                                                    batch_size=32)  # number of images per batch

Found 2120 files belonging to 2 classes.


prefetching + caching

In [55]:
AUTOTUNE = tf.data.AUTOTUNE
training_set = training_set.cache().prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

# Data Augmentation

In [56]:
#This creates different variations of each image during training → helps the CNN generalize.
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
])

## Part 2 - Building the CNN

### Initialising the CNN

In [57]:
cnn = tf.keras.models.Sequential()

In [58]:
cnn.add(tf.keras.Input(shape=(64, 64, 3))) # Add Input layer
cnn.add(data_augmentation)
cnn.add(layers.Rescaling(1./255))

### Step 1 - Convolution

In [68]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 2 - Pooling

In [69]:
cnn.maxpool = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)

### Adding a second convolutional layer

In [70]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [71]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [72]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [73]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

###***add dropout to prevent overfitting

In [74]:
cnn.add(tf.keras.layers.Dropout(0.2))

### Step 5 - Output Layer

In [75]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [76]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [77]:
cnn.fit(x = training_set, validation_data = validation_set, epochs = 25)

Epoch 1/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 84s 320ms/step - accuracy: 0.5034 - loss: 0.6935 - val_accuracy: 0.4717 - val_loss: 0.6937
Epoch 2/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 64s 252ms/step - accuracy: 0.5039 - loss: 0.6932 - val_accuracy: 0.5575 - val_loss: 0.6805
Epoch 3/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.5475 - loss: 0.6847 - val_accuracy: 0.5882 - val_loss: 0.6689
Epoch 4/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.5796 - loss: 0.6703 - val_accuracy: 0.6340 - val_loss: 0.6490
Epoch 5/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.6088 - loss: 0.6598 - val_accuracy: 0.6618 - val_loss: 0.6191
Epoch 6/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.6293 - loss: 0.6413 - val_accuracy: 0.6722 - val_loss: 0.6096
Epoch 7/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.6311 - loss: 0.6401 - val_accuracy: 0.6778 - val_loss: 0.6002
Epoch 8/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 62s 245ms/step - accuracy: 0.6401 - loss: 0

## Part 4 - Making a single prediction

### import single prediction dataset

In [93]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image_A = tf.keras.utils.load_img('/content/drive/MyDrive/CNN/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image_A = tf.keras.utils.img_to_array(test_image_A)
test_image_A = np.expand_dims(test_image_A, axis = 0)
resultA = cnn.predict(test_image_A)
# Use the stored class_names instead of accessing training_set.class_indices
if resultA[0][0] > 0.5:
  prediction_A = class_names[1] # Assuming class_names[1] is 'dog' and class_names[0] is 'cat' based on directory loading
else:
  prediction_A = class_names[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [94]:
print(prediction_A)

dogs


In [95]:
test_image_B = tf.keras.utils.load_img('/content/drive/MyDrive/CNN/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image_B = tf.keras.utils.img_to_array(test_image_B)
test_image_B = np.expand_dims(test_image_B, axis = 0)
resultB = cnn.predict(test_image_B)
# Use the stored class_names instead of accessing training_set.class_indices
if resultB[0][0] > 0.5:
  prediction_B = class_names[1] # Assuming class_names[1] is 'dog' and class_names[0] is 'cat' based on directory loading
else:
  prediction_B = class_names[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [96]:
print(prediction_B)

cats


In [97]:
test_image_C = tf.keras.utils.load_img('/content/drive/MyDrive/CNN/dataset/single_prediction/cat_or_dog_3.jpg', target_size = (64, 64))
test_image_C = tf.keras.utils.img_to_array(test_image_C)
test_image_C = np.expand_dims(test_image_C, axis = 0)
resultC = cnn.predict(test_image_C)
# Use the stored class_names instead of accessing training_set.class_indices
if resultC[0][0] > 0.5:
  prediction_C = class_names[1] # Assuming class_names[1] is 'dog' and class_names[0] is 'cat' based on directory loading
else:
  prediction_C = class_names[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [98]:
print(prediction_C)

dogs
